# Lack of evidence for attention-like limits on accuracy of convolutional neural networks performing a visual search task

## Abstract
What limits our ability to find what we are looking for in the cluttered noisy world? To investigate this, cognitive scientists have long used visual search. In spite of hundreds of studies, it remains unclear how to relate effects found using the discrete item display search task to computations in the visual system. A separate thread of research has studied the visual system of humans and other primates using convolutional neural networks (CNNs) as models. Multiple lines of evidence suggest that training CNNs to perform tasks such as image classification causes them to learn  representations similar to those used by the visual system. These studies raise the question of whether CNNs that have learned such representations behave similarly to humans performing other vision-based tasks. Here we address this by measuring the behavior of CNNs trained for image classification while they perform the discrete item display search task. We first show how a fine-tuning approach often used to adapt pre-trained CNNs to new tasks can produce models that show human-like limitations on this task. However we then demonstrate that we can greatly reduce these effects by changing training,without changing the learned representations. Lastly we show that accuracy is not impaired when single networks are trained to discriminate multiple types of visual search stimuli. Based on these findings, we suggest that CNNs are not necessarily subject to the same limitations as the primate visual system.

## Introduction

### The discrete item display search task
On each trial, subjects view a display (example images, $\mathbf{a}$) of discrete items on a flat background. Subjects respond whether the target is present or absent (condition varies across rows of $\mathbf{a}$). Another condition that varies across trials is the set size, that is, the total number of targets and distractors (varies across rows in $\mathbf{a}$). Many studies focus on *set size effects*. Typical effects are an increase in reaction time or a decrease in accuracy as set size increases (depicted schematically in $\mathbf{b}$, redrawn from (Wolfe et al., 2010) and (Eckstein, 1998). Effects vary based on the features that distinguish targets from distractors (shown in columns). Details in introduction. Accuracy for spatial configuration-type stimuli not shown in $\mathbf{b}$ because this has been less studied (but see E. M. Palmer et al., 2011)

![figure 1](../static/fig1/fig1.png)

## Results
### Human-like limits on accuracy of convolutional neural networks (CNNs) performing the discrete item display search task

When using weights pre-trained on ImageNet in convolutional layers, and fine-tuning weights in fully-connected layers so AlexNet could perform the task, this CNN showed decreases in accuracy as set size increased ($\mathbf{a}$). This decrease was smallest for feature search stimuli (left plot), intermediate for conjunction stimuli (middle plot), and largest for spatial configuration stimuli (right plot). The same approach produced similar results with the VGG16 architecture ($\mathbf{b}$)

![Figure 2](../static/fig2/fig2.png)

### Changing training greatly reduces set size effects
Training histories showed that the accuracy of models trained with the fine tuning approach did not converge
on some asymptotic value, and varied depending on the set size of the search stimuli ($\textbf{a}$, left plot). Increasing the learning rate and including more examples of stimuli with larger set sizes greatly sped up convergence ($\textbf{a}$, right plot). AlexNet models trained with this higher learning rate and larger training set showed reduced set size effects ($\textbf{b}$).
![figure 3](../static/fig3/fig3.png)

### Training single networks on multiple stimuli does not impair accuracy
Two instances of AlexNet trained on datasets containing all three types of visual search stimuli used in this study still attained high accuracy.
![figure 4](../static/fig4/fig4.png)

## Methods
Fine-tuning Alexnet and VGG16:
* based on Põder 2017: https://arxiv.org/pdf/1707.09775.pdf
* Stochastic gradient descent
* Base learning rate: 1e-20
* Rate for fully-connected layers: 0.0001
* epochs: 200

Training  
* Freeze convolutional layers
* Rate for fully-connected layers: 0.001
* Epochs: until validation accuracy did not improve for more than 20 epochs

## References

## Acknowledgments
- Research funded by the Lifelong Learning Machines program, DARPA/Microsystems Technology Office,  DARPA cooperative agreement HR0011-18-2-0019
- David Nicholson was partially supported by the 2017 William K. and Katherine W. Estes Fund to F. Pestilli, R. Goldstone and L. Smith, Indiana University Bloomington.